In [ ]:
import sys, time, os, asyncio, glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from astropy.time import Time, TimeDelta
from lsst.summit.utils.utils import dayObsIntToString
from lsst_efd_client import EfdClient
from lsst.summit.utils.tmaUtils import TMAEventMaker, plotEvent, getAzimuthElevationDataForEvent
%matplotlib inline

In [ ]:
def filterBadValues(values, maxDelta=0.1, maxConsecutiveValues=3):
    """Filter out bad values from a dataset, replacing them in-place.

    This function replaces non-physical points in the dataset with an
    extrapolation of the preceding two values. No more than 3 successive data
    points are allowed to be replaced. Minimum length of the input is 3 points.

    Parameters
    ----------
    values : `list` or `np.ndarray`
        The dataset containing the values to be filtered.
    maxDelta : `float`, optional
        The maximum allowed difference between consecutive values. Values with
        a difference greater than `maxDelta` will be considered as bad values
        and replaced with an extrapolation.
    maxConsecutiveValues : `int`, optional
        The maximum number of consecutive values to replace. Defaults to 3.

    Returns
    -------
    nBadPoints : `int`
        The number of bad values that were replaced out.
    """
    # Find non-physical points and replace with extrapolation. No more than
    # maxConsecutiveValues successive data points can be replaced.
    badCounter = 0
    consecutiveCounter = 0

    log = logging.getLogger(__name__)

    median = np.nanmedian(values)
    # if either of the the first two points are more than maxDelta away from
    # the median, replace them with the median
    for i in range(2):
        if abs(values[i] - median) > maxDelta:
            log.warning(f"Replacing bad value of {values[i]} at index {i} with {median=}")
            values[i] = median
            badCounter += 1

    # from the second element of the array, walk through and calculate the
    # difference between each element and the previous one. If the difference
    # is greater than maxDelta, replace the element with the average of the
    # previous two known good values, i.e. ones which have not been replaced.
    # if the first two points differ from the median by more than maxDelta,
    # replace them with the median
    lastGoodValue1 = values[1]  # the most recent good value
    lastGoodValue2 = values[0]  # the second most recent good value
    replacementValue = (lastGoodValue1 + lastGoodValue2) / 2.0  # in case we have to replace the first value
    for i in range(2, len(values)):
        if abs(values[i] - lastGoodValue1) >= maxDelta:
            if consecutiveCounter < maxConsecutiveValues:
                consecutiveCounter += 1
                badCounter += 1
                log.warning(f"Replacing value at index {i} with {replacementValue}")
                values[i] = replacementValue
            else:
                log.warning(
                    f"More than 3 consecutive replacements at index {i}. Stopping replacements"
                    " until the next good value."
                )
        else:
            lastGoodValue2 = lastGoodValue1
            lastGoodValue1 = values[i]
            replacementValue = (lastGoodValue1 + lastGoodValue2) / 2.0
            consecutiveCounter = 0
    return badCounter


In [ ]:
client = EfdClient("usdf_efd")
eventMaker = TMAEventMaker()

In [ ]:
dayObs = 20240404
dayObsString = dayObsIntToString(dayObs)
filename = f'/home/c/cslage/u/MTMount/json_metadata/startracker_narrow_{dayObsString}.json'
jsonData = pd.read_json(filename).T
jsonData = jsonData.sort_index()

In [ ]:
jsonData.columns

In [ ]:
seqNums = [[23,82],[83,142],[143,202],[203,262],[263,322],[323,382],\
    [383,442],[443,502],[503,562],[563,622],[623,646],[648,657],[659,718],\
    [719,778],[779,830],[832,911],[1044,1075]]
len(seqNums)

In [ ]:
pdf = PdfPages("/home/c/cslage/u/MTMount/mount_plots/Long_Term_Tracking_04Apr24.pdf")
fig = plt.figure(figsize=(10,6))

for [firstSeqNum, lastSeqNum] in seqNums:
    times = []
    ras = []
    decs = []
    for seqNum in range(firstSeqNum, lastSeqNum):
        try:
            timeInput = dayObsString + 'T' + jsonData.iloc[seqNum]['UTC narrow']
            time = Time(timeInput, scale='utc').unix_tai
            ra = jsonData.iloc[seqNum]['Calculated Ra narrow']
            dec = jsonData.iloc[seqNum]['Calculated Dec narrow']
            ras.append(ra)
            decs.append(dec)
            times.append(time)
        except:
            continue
    times = np.array(times)
    times -= times[0]
    ras = np.array(ras)
    ras = (ras - ras[0]) * 3600.0
    decs = np.array(decs)
    decs = (decs - decs[0]) * 3600.0
    raDrift = (ras[-1] - ras[0]) / times[-1] * 30.0
    decDrift = (decs[-1] - decs[0]) / times[-1] * 30.0
    axs = fig.subplots(1,2)
    plt.suptitle(f"Long term tracking drift {dayObsString}: {firstSeqNum} - {lastSeqNum}  RA = {ra:.2f}, Dec = {dec:.2f}", \
                 fontsize=18)
    plt.subplots_adjust(wspace=0.3)
    axs[0].set_title(f"RA drift = {raDrift:.2f} arcseconds/30 seconds")
    axs[0].scatter(times, ras)
    axs[0].set_ylabel("RA Drift (arcseconds)")
    axs[0].set_xlabel("Time (seconds)")
    axs[1].set_title(f"Dec drift = {decDrift:.2f} arcseconds/30 seconds")
    axs[1].scatter(times, decs)
    axs[1].set_ylabel("Dec Drift (arcseconds)")
    axs[1].set_xlabel("Time (seconds)")
    pdf.savefig(fig)  # saves the current figure into a pdf page
    plt.clf()
pdf.close()




In [ ]:
prePad = 10.0
postPad = 10.0
pdf = PdfPages("/home/c/cslage/u/MTMount/mount_plots/Long_Term_Tracking_Events_04Apr24.pdf")
fig = plt.figure(figsize=(15,6))

events = eventMaker.getEvents(dayObs)
longTracks = [e for e in events if (e.type.name == 'TRACKING' and e.duration > 500)]

for n, track in enumerate(longTracks):
    if n < 2 or n == 19:
        continue
    seqNums = []
    for i in range(len(jsonData)):
        try:
            time = Time(jsonData.iloc[i]['MJD narrow'], format='mjd').unix_tai - 37.0
            if time > track.begin.value + prePad and time < track.end.value - postPad:
                seqNums.append(i)
        except:
            continue
    print(n, len(seqNums))
    if len(seqNums) == 0:
        continue
    azimuthData, elevationData = getAzimuthElevationDataForEvent(client, track)
    azTimes = azimuthData['timestamp'].values
    azTimes -= azTimes[0]
    azErrors = azimuthData['azError'].values
    filterBadValues(azErrors, maxConsecutiveValues=10)
    elTimes = elevationData['timestamp'].values
    elTimes -= elTimes[0]
    elErrors = elevationData['elError'].values
    filterBadValues(elErrors, maxConsecutiveValues=10)


    times = []
    ras = []
    decs = []
    for seqNum in seqNums:
        try:
            time = Time(jsonData.iloc[seqNum]['MJD narrow'], format='mjd').unix_tai
            ra = jsonData.iloc[seqNum]['Calculated Ra narrow']
            dec = jsonData.iloc[seqNum]['Calculated Dec narrow']
            ras.append(ra)
            decs.append(dec)
            times.append(time)
        except:
            continue
    times = np.array(times)
    times -= times[0]
    ras = np.array(ras)
    ras = (ras - ras[0]) * 3600.0
    decs = np.array(decs)
    decs = (decs - decs[0]) * 3600.0
    raDrift = (ras[-1] - ras[0]) / times[-1] * 30.0
    decDrift = (decs[-1] - decs[0]) / times[-1] * 30.0
    axs = fig.subplots(1,3)
    plt.suptitle(f"Long term tracking drift {dayObsString}: {seqNums[0]} - {seqNums[-1]}  RA = {ra:.2f}, Dec = {dec:.2f}", \
                 fontsize=18)
    plt.subplots_adjust(wspace=0.3)
    axs[0].set_title(f"RA drift = {raDrift:.2f} arcseconds/30 seconds")
    axs[0].scatter(times, ras)
    axs[0].set_ylabel("RA Drift (arcseconds)")
    axs[0].set_xlabel("Time (seconds)")
    axs[1].set_title(f"Dec drift = {decDrift:.2f} arcseconds/30 seconds")
    axs[1].scatter(times, decs)
    axs[1].set_ylabel("Dec Drift (arcseconds)")
    axs[1].set_xlabel("Time (seconds)")
    axs[2].set_title("Encoder tracking errors")
    axs[2].plot(azTimes, azErrors, color='green', label='Az')
    axs[2].plot(elTimes, elErrors, color='red', label='El')
    axs[2].set_ylabel("Errors (arcsec)")
    axs[2].set_xlabel("Time (sec)")
    axs[2].set_ylim(-1.0,1.0)
    axs[2].legend()
    pdf.savefig(fig)  # saves the current figure into a pdf page
    plt.clf()
pdf.close()
